In [1]:
import pandas as pd
import numpy as np
import json

### Load Data

In [2]:
column_lookup = {}
with open("./data/reference/column_lookup.txt") as text_file:
    for line in text_file:
        item = json.loads(line)
        column_lookup[item['column']] = item['meaning']

In [3]:
df_student = pd.read_csv("./data/2_merged/student.csv")

In [4]:
df_student.shape

(612004, 32)

In [5]:
df_student.CNTSCHID.nunique()

21903

In [6]:
df_school = pd.read_csv("./data/2_merged/school.csv")

In [7]:
df_teacher = pd.read_csv("./data/2_merged/teacher.csv")

### Teacher dataframe prepare for merge

In [8]:
# Teacher columns
for column in df_teacher.columns:
    print('{:<14} -> {}'.format(column, column_lookup[column]))

CNTRYID        -> Country Identifier
CNT            -> Country code 3-character
CNTSCHID       -> Intl. School ID
CNTTCHID       -> Intl. Teacher ID
TEACHERID      -> Teacher identification code
STRATUM        -> Stratum ID 7-character (cnt + region ID + original stratum ID)
TC001Q01NA     -> Are you female or male?
TC002Q01NA     -> How old are you?
TC005Q01NA     -> What is your current employment status as a teacher? My employment status at this school
TC007Q01NA     -> How many years of work experience do you have? Year(s) working as a teacher at this school
TC186Q01HA     -> Country of birth
tch_satisfied  -> NEW: Does teacher find the work meaningful (higher the better)


In [9]:
# satisfied, age, # years as teacher
df_sum_teacher = df_teacher.groupby(['CNTSCHID','CNTRYID'])[['tch_satisfied', 'TC002Q01NA', 'TC007Q01NA']].mean().reset_index()

In [10]:
df_sum_teacher = df_sum_teacher.rename(columns={
    "tch_satisfied": "mean_teacher_satisfied",
    "TC002Q01NA": "mean_age_of_teachers",
    "TC007Q01NA": "mean_yaer_experiance"
})

In [11]:
df_sum_teacher.head()

,CNTSCHID,CNTRYID,mean_teacher_satisfied,mean_age_of_teachers,mean_yaer_experiance
0,800002.0,8.0,18.000000,47.888889,7.888889
1,800003.0,8.0,15.200000,42.400000,9.200000
2,800004.0,8.0,16.111111,36.375000,5.555556
3,800005.0,8.0,18.800000,40.285714,14.428571
4,800006.0,8.0,18.625000,39.687500,12.937500


### School dataframe prepare for merge

In [12]:
df_school.shape

(21903, 16)

In [13]:
# school columns
for column in df_school.columns:
    print('{:<23} -> {}'.format(column, column_lookup[column]))

CNTRYID                 -> Country Identifier
CNT                     -> Country code 3-character
CNTSCHID                -> Intl. School ID
CYC                     -> PISA Assessment Cycle (2 digits + 2 character Assessment type - MS/FT)
NatCen                  -> National Centre 6-digit Code
Region                  -> Region
STRATUM                 -> Stratum ID 7-character (cnt + region ID + original stratum ID)
SC001Q01TA              -> Which of the following definitions best describes the community in which your school is located?
SC013Q01TA              -> Is your school a public or a private school?
SC011Q01TA              -> Which of the following statements best describes the schooling available to students in your location?
SC002Q01TA              -> As of <February 1, 2018>, what was the total school enrolment (number of students)? Number of boys
SC002Q02TA              -> As of <February 1, 2018>, what was the total school enrolment (number of students)? Number of girls
SC

In [14]:
# schoolid, public/private, class size,
df_sum_school = df_school[['CNTRYID', 'CNTSCHID', 'SC013Q01TA', 'SC003Q01TA', 'education_obstacle', 'problematic_students', 'degree_score_teachers']]

In [15]:
df_sum_school.head()

,CNTRYID,CNTSCHID,SC013Q01TA,SC003Q01TA,education_obstacle,problematic_students,degree_score_teachers
0,8.0,800002.0,1.0,2.0,11.0,8.0,0.519231
1,8.0,800003.0,1.0,1.0,9.0,21.0,NaN
2,8.0,800004.0,1.0,2.0,8.0,10.0,2.928571
3,8.0,800005.0,1.0,3.0,14.0,11.0,3.371429
4,8.0,800006.0,1.0,6.0,6.0,8.0,0.750000


### Student dataframe prepare for merge

In [16]:
# dropping OECD contry and birth month columns
df_student.drop(columns=['OECD', 'ST003D02T'], inplace=True, axis=1)
df_student.shape

(612004, 30)

In [17]:
# Teacher columns
for column in df_student.columns:
    print('{:<20} -> {}'.format(column, column_lookup[column]))

CNTRYID              -> Country Identifier
CNT                  -> Country code 3-character
CNTSCHID             -> Intl. School ID
CNTSTUID             -> Intl. Student ID
STRATUM              -> Stratum ID 7-character (cnt + region ID + original stratum ID)
ST001D01T            -> Student International Grade (Derived)
ST003D03T            -> Student (Standardized) Birth -Year
ST004D01T            -> Student (Standardized) Gender
PA042Q01TA           -> What is your annual household income?
ESCS                 -> Index of economic, social and cultural status
WEALTH               -> Wealth
PV1MATH              -> Plausible Value 1 in Mathematics
PV1READ              -> Plausible Value 1 in Reading
PV1SCIE              -> Plausible Value 1 in Science
PV1RCLI              -> Plausible Value 1 in Cognitive Process Subscale of Reading - Locate Information
PV1RCUN              -> Plausible Value 1 in Cognitive Process Subscale of Reading - Understand
PV1RCER              -> Plausible Value

### Merge dataframes

In [18]:
df_student.shape

(612004, 30)

In [19]:
df_sum_school.shape

(21903, 7)

In [20]:
df_final = pd.merge(df_student, df_sum_school, on=["CNTSCHID","CNTRYID"], how="outer")
df_final.shape

(612004, 35)

In [21]:
df_final = pd.merge(df_final, df_sum_teacher, on=["CNTSCHID","CNTRYID"], how="outer")
df_final.shape

(612004, 38)

In [22]:
pd.get_option("display.max_columns")


20

In [23]:
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_rows', 40)

In [24]:
df_final.sample(10)

,CNTRYID,CNT,CNTSCHID,CNTSTUID,STRATUM,ST001D01T,ST003D03T,ST004D01T,PA042Q01TA,ESCS,WEALTH,PV1MATH,PV1READ,PV1SCIE,PV1RCLI,PV1RCUN,PV1RCER,math_score,math_answered,reading_score,reading_answered,science_score,science_answered,home_art,bad_home_conditions,school_bad_time,emotional_status,family_hi_ed,quality_tch,reading_not_easy,SC013Q01TA,SC003Q01TA,education_obstacle,problematic_students,degree_score_teachers,mean_teacher_satisfied,mean_age_of_teachers,mean_yaer_experiance
319777,442.0,LUX,44200036.0,44205609.0,LUX0101,10.0,2002.0,1.0,6.0,1.0647,0.8925,576.317,740.079,668.413,629.114,639.622,646.659,110.0,24.0,530.0,56.0,0.0,0.0,9.0,0.0,0.0,0.326087,6.0,1.0,4.0,1.0,3.0,9.0,13.0,4.660000,NaN,NaN,NaN
203485,300.0,GRC,30000063.0,30004338.0,GRC0003,8.0,2002.0,2.0,NaN,-1.7070,-0.6586,340.295,235.675,324.431,260.513,231.278,248.723,0.0,0.0,200.0,53.0,0.0,0.0,4.0,2.0,1.0,0.847826,2.0,5.0,4.0,1.0,3.0,5.0,11.0,NaN,NaN,NaN,NaN
9039,31.0,QAZ,3100061.0,3101086.0,QAZ0204,9.0,2002.0,2.0,NaN,-1.5716,-1.6272,449.599,414.523,416.513,308.124,334.603,293.464,70.0,22.0,280.0,48.0,0.0,0.0,0.0,0.0,6.0,NaN,2.0,5.0,5.0,1.0,3.0,7.0,7.0,NaN,17.263158,35.238095,9.285714
496036,724.0,ESP,72400018.0,72401551.0,ESP0305,9.0,2002.0,1.0,NaN,-0.1921,-0.4487,304.428,350.919,348.570,365.829,338.628,303.047,20.0,8.0,300.0,49.0,60.0,15.0,2.0,1.0,0.0,0.673913,5.0,2.0,4.0,1.0,2.0,14.0,12.0,NaN,15.055556,48.333333,9.111111
224094,360.0,IDN,36000158.0,36012718.0,IDN0101,9.0,2002.0,1.0,NaN,-1.0856,-2.8136,380.090,293.733,401.881,299.146,295.758,302.770,0.0,0.0,240.0,56.0,0.0,0.0,4.0,2.0,0.0,0.934783,0.0,4.0,3.0,1.0,2.0,12.0,9.0,1.410256,NaN,NaN,NaN
537780,784.0,ARE,78400454.0,78414715.0,ARE0109,10.0,2002.0,1.0,NaN,1.2389,0.5604,448.906,425.784,393.940,415.947,329.447,422.554,60.0,22.0,310.0,56.0,0.0,0.0,4.0,0.0,0.0,NaN,13.0,4.0,2.0,2.0,4.0,6.0,8.0,3.285000,17.666667,45.400000,12.434783
378130,578.0,NOR,57800168.0,57806776.0,NOR0001,10.0,2002.0,1.0,NaN,NaN,NaN,442.211,457.235,365.098,515.598,475.602,474.949,50.0,24.0,390.0,57.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,3.0,6.0,12.0,NaN,NaN,NaN,NaN
203801,344.0,HKG,34400051.0,34400807.0,HKG0002,10.0,2002.0,1.0,6.0,0.1249,-0.8965,493.458,449.842,484.092,406.396,452.361,461.103,60.0,21.0,360.0,51.0,0.0,0.0,4.0,2.0,17.0,0.326087,4.0,1.0,3.0,2.0,4.0,11.0,9.0,4.000000,13.380952,36.095238,10.666667
185234,250.0,FRA,25000024.0,25003342.0,FRA0203,10.0,2002.0,2.0,NaN,-0.3899,0.7128,514.807,482.369,501.660,507.803,488.060,509.847,140.0,21.0,430.0,58.0,0.0,0.0,3.0,0.0,0.0,0.760870,0.0,5.0,5.0,1.0,3.0,13.0,17.0,NaN,NaN,NaN,NaN
359714,504.0,MAR,50400164.0,50405608.0,MAR0004,9.0,2002.0,2.0,NaN,-0.1405,0.2361,359.963,318.778,326.194,242.261,269.418,251.766,0.0,0.0,190.0,47.0,50.0,35.0,1.0,3.0,0.0,0.760870,5.0,4.0,4.0,1.0,6.0,14.0,21.0,0.230769,19.333333,51.187500,3.733333


### Explore

In [25]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 612004 entries, 0 to 612003
Data columns (total 38 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   CNTRYID                 612004 non-null  float64
 1   CNT                     612004 non-null  object 
 2   CNTSCHID                612004 non-null  float64
 3   CNTSTUID                612004 non-null  float64
 4   STRATUM                 612004 non-null  object 
 5   ST001D01T               612004 non-null  float64
 6   ST003D03T               612004 non-null  float64
 7   ST004D01T               612002 non-null  float64
 8   PA042Q01TA              67913 non-null   float64
 9   ESCS                    597625 non-null  float64
 10  WEALTH                  598267 non-null  float64
 11  PV1MATH                 606627 non-null  float64
 12  PV1READ                 606627 non-null  float64
 13  PV1SCIE                 606627 non-null  float64
 14  PV1RCLI             

In [26]:
column_lookup['PA042Q01TA']

'What is your annual household income?'

In [27]:
df_final.duplicated().sum()

0

In [28]:
df_final.CNTSCHID.nunique()

21903

- unique student id should be country + studentId

In [33]:
df_final.describe()

,CNTRYID,CNTSCHID,CNTSTUID,ST001D01T,ST003D03T,ST004D01T,PA042Q01TA,ESCS,WEALTH,PV1MATH,PV1READ,PV1SCIE,PV1RCLI,PV1RCUN,PV1RCER,math_score,math_answered,reading_score,reading_answered,science_score,science_answered,home_art,bad_home_conditions,school_bad_time,emotional_status,family_hi_ed,quality_tch,reading_not_easy,SC013Q01TA,SC003Q01TA,education_obstacle,problematic_students,degree_score_teachers,mean_teacher_satisfied,mean_age_of_teachers,mean_yaer_experiance
count,612004.000000,6.120040e+05,6.120040e+05,612004.000000,612004.000000,612002.000000,67913.000000,597625.000000,598267.000000,606627.000000,606627.000000,606627.000000,551930.000000,551930.000000,551930.000000,606627.000000,606627.000000,606627.000000,606627.00000,606627.000000,606627.000000,612004.000000,612004.000000,612004.000000,493202.000000,612004.000000,612004.000000,612004.000000,552772.000000,547272.000000,580015.000000,580082.000000,367167.000000,154500.000000,154519.000000,154517.000000
mean,448.931030,4.489354e+07,4.490023e+07,11.077864,2002.065446,1.501704,3.136130,-0.280831,-0.431981,461.220395,456.123004,460.694397,459.682556,460.561792,463.577117,41.706535,9.306081,353.154410,50.96847,71.515338,14.902154,3.006786,0.842957,1.284756,0.694637,2.551103,2.486846,2.943095,1.196631,4.155212,9.755382,12.908263,2.561386,16.848956,43.249365,9.620907
std,269.865482,2.698691e+07,2.698768e+07,11.046151,0.247740,0.499998,1.925627,1.115985,1.240680,104.359230,108.047528,102.664640,113.182050,110.357085,113.436604,56.118004,10.213900,122.518369,11.11526,93.849470,16.490148,2.270446,1.264016,2.617000,0.225618,2.918103,2.194162,1.679572,0.397451,1.982413,3.487602,3.920094,1.500569,1.889475,4.951974,4.735149
min,8.000000,8.000020e+05,8.000010e+05,7.000000,2001.000000,1.000000,1.000000,-8.173400,-7.546500,24.743000,0.000000,58.736000,0.000000,43.117000,24.794000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.065217,0.000000,0.000000,0.000000,1.000000,1.000000,5.000000,6.000000,0.000000,8.000000,26.000000,0.000000
25%,203.000000,2.030021e+07,2.030481e+07,9.000000,2002.000000,1.000000,1.000000,-1.009500,-1.135300,386.656000,378.104500,385.602000,379.255000,380.577000,380.328000,0.000000,0.000000,280.000000,49.00000,0.000000,0.000000,1.000000,0.000000,0.000000,0.586957,0.000000,1.000000,2.000000,1.000000,3.000000,7.000000,10.000000,1.596154,15.608696,39.941176,6.090909
50%,422.000000,4.220031e+07,4.220616e+07,10.000000,2002.000000,2.000000,3.000000,-0.187600,-0.370800,460.659000,455.658000,458.200000,462.020000,460.642000,462.760500,0.000000,0.000000,370.000000,55.00000,0.000000,0.000000,3.000000,0.000000,0.000000,0.760870,2.000000,2.000000,3.000000,1.000000,4.000000,9.000000,12.000000,2.152174,16.833333,43.285714,9.000000
75%,703.000000,7.030015e+07,7.030307e+07,10.000000,2002.000000,2.000000,5.000000,0.594000,0.322200,535.285000,533.452000,533.442000,541.175500,540.299000,545.167750,75.000000,21.000000,450.000000,58.00000,130.000000,34.000000,4.000000,1.000000,2.000000,0.847826,4.000000,4.000000,4.000000,1.000000,5.000000,12.000000,15.000000,3.582418,18.055556,46.550000,12.523810
max,983.000000,9.830081e+07,9.832912e+07,98.000000,2003.000000,2.000000,6.000000,4.205100,4.753400,888.064000,887.692000,886.081000,889.585000,882.849000,962.138000,240.000000,28.000000,720.000000,97.00000,400.000000,47.000000,9.000000,7.000000,21.000000,0.934783,16.000000,13.000000,11.000000,2.000000,9.000000,20.000000,24.000000,14.890110,23.777778,70.000000,50.000000


In [34]:
# Teacher columns
for column in df_final.columns:
    print('{:<24} -> {}'.format(column, column_lookup[column]))

CNTRYID                  -> Country Identifier
CNT                      -> Country code 3-character
CNTSCHID                 -> Intl. School ID
CNTSTUID                 -> Intl. Student ID
STRATUM                  -> Stratum ID 7-character (cnt + region ID + original stratum ID)
ST001D01T                -> Student International Grade (Derived)
ST003D03T                -> Student (Standardized) Birth -Year
ST004D01T                -> Student (Standardized) Gender
PA042Q01TA               -> What is your annual household income?
ESCS                     -> Index of economic, social and cultural status
WEALTH                   -> Wealth
PV1MATH                  -> Plausible Value 1 in Mathematics
PV1READ                  -> Plausible Value 1 in Reading
PV1SCIE                  -> Plausible Value 1 in Science
PV1RCLI                  -> Plausible Value 1 in Cognitive Process Subscale of Reading - Locate Information
PV1RCUN                  -> Plausible Value 1 in Cognitive Process Subscale

In [35]:
df_final= df_final.rename(columns={
"CNTRYID": "country_id",
"CNT":"country_code",
"CNTSCHID": "school_id",
"CNTSTUID": "student_id",
"STRATUM": "social_level_id",
"ST001D01T": "grade",
"ST003D03T": "birth_year",
"ST004D01T": "gender",
"PA042Q01TA": "income",
"ESCS": "socio_economic_index",
"WEALTH": "wealth",
"PV1MATH": "score_math",
"PV1READ": "score_reading",
"PV1SCIE": "score_science",
"PV1RCLI": "score_find_info",
"PV1RCUN": "score_understand",
"PV1RCER": "score_evaluate",
"home_art": "home_art_score",
"bad_home_conditions": "bad_home_conditions",
"school_bad_time": "bullied_at_school",
"emotional_status": "emotional_status",
"family_hi_ed": "family_high_edu",
"quality_tch": "teacher_quality_stu",
"reading_not_easy": "reading_not_easy",
"SC013Q01TA": "school_type",
"SC003Q01TA": "class_size",
"math_score": "cust_math_score",
"math_answered": "num_math_answered",
"reading_score": "cust_reading_score",
"reading_answered": "num_reading_answered",
"science_score": "cust_science_score",
"science_answered": "num_science_answered",
})

In [36]:
df_final.head()

,country_id,country_code,school_id,student_id,social_level_id,grade,birth_year,gender,income,socio_economic_index,wealth,score_math,score_reading,score_science,score_find_info,score_understand,score_evaluate,cust_math_score,num_math_answered,cust_reading_score,num_reading_answered,cust_science_score,num_science_answered,home_art_score,bad_home_conditions,bullied_at_school,emotional_status,family_high_edu,teacher_quality_stu,reading_not_easy,school_type,class_size,education_obstacle,problematic_students,degree_score_teachers,mean_teacher_satisfied,mean_age_of_teachers,mean_yaer_experiance
0,8.0,ALB,800115.0,800001.0,ALB0107,10.0,2002.0,1.0,NaN,-0.2154,-0.5795,432.285,341.838,390.471,354.662,370.396,395.945,100.0,22.0,320.0,52.0,0.0,0.0,4.0,0.0,1.0,0.673913,6.0,3.0,4.0,1.0,4.0,8.0,10.0,1.0,19.533333,32.8,6.466667
1,8.0,ALB,800115.0,800028.0,ALB0107,10.0,2002.0,2.0,NaN,-1.9261,-2.5475,489.429,366.323,410.723,310.592,369.251,351.639,35.0,11.0,300.0,55.0,0.0,0.0,1.0,0.0,2.0,0.760870,1.0,4.0,2.0,1.0,4.0,8.0,10.0,1.0,19.533333,32.8,6.466667
2,8.0,ALB,800115.0,800042.0,ALB0107,10.0,2002.0,1.0,NaN,-0.9100,-1.1279,440.172,389.501,460.947,354.064,367.586,389.140,0.0,0.0,390.0,55.0,0.0,0.0,0.0,0.0,1.0,0.934783,1.0,4.0,2.0,1.0,4.0,8.0,10.0,1.0,19.533333,32.8,6.466667
3,8.0,ALB,800115.0,800170.0,ALB0107,10.0,2002.0,1.0,NaN,-0.7399,-1.7221,498.720,290.116,362.285,214.095,230.932,278.721,0.0,0.0,190.0,47.0,40.0,15.0,2.0,1.0,2.0,0.934783,4.0,2.0,2.0,1.0,4.0,8.0,10.0,1.0,19.533333,32.8,6.466667
4,8.0,ALB,800115.0,800189.0,ALB0107,10.0,2002.0,1.0,NaN,-1.8613,-1.2863,340.907,394.439,346.622,367.276,379.533,387.001,0.0,0.0,310.0,51.0,100.0,37.0,4.0,2.0,1.0,0.760870,0.0,1.0,2.0,1.0,4.0,8.0,10.0,1.0,19.533333,32.8,6.466667


In [40]:
int_columns = ["country_id","school_id","student_id","grade","birth_year","home_art_score","bad_home_conditions","bullied_at_school","family_high_edu","teacher_quality_stu","reading_not_easy","class_size","education_obstacle","school_type","gender", "cust_math_score","num_math_answered","cust_reading_score","num_reading_answered","cust_science_score","num_science_answered", "problematic_students"]
type_dict = {}
for item in int_columns:
    type_dict[item] = "Int64"

In [41]:
df_final = df_final.astype(type_dict)

In [51]:
# Round test score values
round_columns = ["score_math","score_reading","score_science","score_find_info","score_understand","score_evaluate"]
df_final[round_columns] = df_final[round_columns].round(decimals=0).astype("Int64")

In [52]:
df_final.head()

,country_id,country_code,school_id,student_id,social_level_id,grade,birth_year,gender,income,socio_economic_index,wealth,score_math,score_reading,score_science,score_find_info,score_understand,score_evaluate,cust_math_score,num_math_answered,cust_reading_score,num_reading_answered,cust_science_score,num_science_answered,home_art_score,bad_home_conditions,bullied_at_school,emotional_status,family_high_edu,teacher_quality_stu,reading_not_easy,school_type,class_size,education_obstacle,problematic_students,degree_score_teachers,mean_teacher_satisfied,mean_age_of_teachers,mean_yaer_experiance
0,8,ALB,800115,800001,ALB0107,10,2002,1,NaN,-0.2154,-0.5795,432,342,390,355,370,396,100,22,320,52,0,0,4,0,1,0.673913,6,3,4,1,4,8,10,1.0,19.533333,32.8,6.466667
1,8,ALB,800115,800028,ALB0107,10,2002,2,NaN,-1.9261,-2.5475,489,366,411,311,369,352,35,11,300,55,0,0,1,0,2,0.760870,1,4,2,1,4,8,10,1.0,19.533333,32.8,6.466667
2,8,ALB,800115,800042,ALB0107,10,2002,1,NaN,-0.9100,-1.1279,440,390,461,354,368,389,0,0,390,55,0,0,0,0,1,0.934783,1,4,2,1,4,8,10,1.0,19.533333,32.8,6.466667
3,8,ALB,800115,800170,ALB0107,10,2002,1,NaN,-0.7399,-1.7221,499,290,362,214,231,279,0,0,190,47,40,15,2,1,2,0.934783,4,2,2,1,4,8,10,1.0,19.533333,32.8,6.466667
4,8,ALB,800115,800189,ALB0107,10,2002,1,NaN,-1.8613,-1.2863,341,394,347,367,380,387,0,0,310,51,100,37,4,2,1,0.760870,0,1,2,1,4,8,10,1.0,19.533333,32.8,6.466667


In [53]:
df_final.to_csv("./data/3_cleaned/students.csv",index=False)